In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('cleaned_cars.csv')

df

,url,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,...,body_type,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country
0,https://www.autoscout24.com/offers/alfa-romeo-...,alfa romeo,159,5500.0,233685.0,diesel,red,manual,136.0,1956.0,...,sedan,4.0,5.0,front,134.0,used,grey,NaN,2011.0,it
1,https://www.autoscout24.com/offers/alfa-romeo-...,alfa romeo,159,13990.0,134000.0,gasoline,red,manual,200.0,1742.0,...,sedan,4.0,5.0,front,NaN,used,NaN,NaN,2010.0,es
2,https://www.autoscout24.com/offers/renault-oth...,renault,NaN,2300.0,27000.0,gasoline,yellow,manual,80.0,800.0,...,sedan,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.0,be
3,https://www.autoscout24.com/offers/opel-antara...,opel,antara,3499.0,212575.0,gasoline,black,manual,140.0,2405.0,...,off-road/pick-up,5.0,5.0,4wd,229.0,used,NaN,metallic,2008.0,nl
4,https://www.autoscout24.com/offers/alfa-romeo-...,alfa romeo,giulia,39800.0,7100.0,gasoline,white,automatic,280.0,1995.0,...,sedan,4.0,4.0,front,NaN,employee's car,full leather,black,2023.0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15412,https://www.autoscout24.com/offers/tesla-model...,tesla,model 3,31000.0,24000.0,electric,white,automatic,120.0,NaN,...,sedan,5.0,5.0,rear,NaN,used,full leather,black,2020.0,it
15413,https://www.autoscout24.com/offers/tesla-model...,tesla,model 3,28590.0,49393.0,electric,white,automatic,325.0,NaN,...,sedan,4.0,5.0,rear,NaN,used,NaN,metallic,2020.0,es
15414,https://www.autoscout24.com/offers/tesla-model...,tesla,model y,57900.0,7749.0,electric,black,automatic,534.0,NaN,...,off-road/pick-up,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,nl
15415,https://www.autoscout24.com/offers/tesla-model...,tesla,model y,59900.0,2500.0,electric,grey,NaN,480.0,NaN,...,sedan,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,fr


## Preprocessing

In [3]:
# Drop the url column
df = df.drop('url', axis=1)

df

,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,seller,body_type,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country
0,alfa romeo,159,5500.0,233685.0,diesel,red,manual,136.0,1956.0,dealer,sedan,4.0,5.0,front,134.0,used,grey,NaN,2011.0,it
1,alfa romeo,159,13990.0,134000.0,gasoline,red,manual,200.0,1742.0,dealer,sedan,4.0,5.0,front,NaN,used,NaN,NaN,2010.0,es
2,renault,NaN,2300.0,27000.0,gasoline,yellow,manual,80.0,800.0,dealer,sedan,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.0,be
3,opel,antara,3499.0,212575.0,gasoline,black,manual,140.0,2405.0,dealer,off-road/pick-up,5.0,5.0,4wd,229.0,used,NaN,metallic,2008.0,nl
4,alfa romeo,giulia,39800.0,7100.0,gasoline,white,automatic,280.0,1995.0,dealer,sedan,4.0,4.0,front,NaN,employee's car,full leather,black,2023.0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15412,tesla,model 3,31000.0,24000.0,electric,white,automatic,120.0,NaN,private seller,sedan,5.0,5.0,rear,NaN,used,full leather,black,2020.0,it
15413,tesla,model 3,28590.0,49393.0,electric,white,automatic,325.0,NaN,dealer,sedan,4.0,5.0,rear,NaN,used,NaN,metallic,2020.0,es
15414,tesla,model y,57900.0,7749.0,electric,black,automatic,534.0,NaN,private seller,off-road/pick-up,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,nl
15415,tesla,model y,59900.0,2500.0,electric,grey,NaN,480.0,NaN,dealer,sedan,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,fr


In [4]:
# Check if model column has any missing values
print(df['model'].isnull().sum())

# Replace the missing values with the most frequent value for the corresponding brand
df['model'] = df.groupby('brand')['model'].transform(lambda x: x.fillna(x.mode()[0]))

print(df['model'].isnull().sum())

219
0


In [13]:
# Use a unique id for each model (brand + model)
df['model'] = df['brand'] + '_' + df['model']

df

,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,seller,...,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country,model_id
0,alfa romeo,alfa romeo_159,5500.0,233685.0,diesel,red,manual,136.0,1956.0,dealer,...,4.0,5.0,front,134.0,used,grey,NaN,2011.0,it,alfa romeo_159
1,alfa romeo,alfa romeo_159,13990.0,134000.0,gasoline,red,manual,200.0,1742.0,dealer,...,4.0,5.0,front,NaN,used,NaN,NaN,2010.0,es,alfa romeo_159
2,renault,renault_clio,2300.0,27000.0,gasoline,yellow,manual,80.0,800.0,dealer,...,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.0,be,renault_clio
3,opel,opel_antara,3499.0,212575.0,gasoline,black,manual,140.0,2405.0,dealer,...,5.0,5.0,4wd,229.0,used,NaN,metallic,2008.0,nl,opel_antara
4,alfa romeo,alfa romeo_giulia,39800.0,7100.0,gasoline,white,automatic,280.0,1995.0,dealer,...,4.0,4.0,front,NaN,employee's car,full leather,black,2023.0,de,alfa romeo_giulia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15412,tesla,tesla_model 3,31000.0,24000.0,electric,white,automatic,120.0,NaN,private seller,...,5.0,5.0,rear,NaN,used,full leather,black,2020.0,it,tesla_model 3
15413,tesla,tesla_model 3,28590.0,49393.0,electric,white,automatic,325.0,NaN,dealer,...,4.0,5.0,rear,NaN,used,NaN,metallic,2020.0,es,tesla_model 3
15414,tesla,tesla_model y,57900.0,7749.0,electric,black,automatic,534.0,NaN,private seller,...,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,nl,tesla_model y
15415,tesla,tesla_model y,59900.0,2500.0,electric,grey,NaN,480.0,NaN,dealer,...,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,fr,tesla_model y


In [5]:
# Check if price column has any missing values
print(df['price'].isnull().sum())

# Check if price column has any zero values
print(df['price'].eq(0).sum())

0
0


In [6]:
# Check if mileage column has any missing values
print(df['mileage'].isnull().sum())

# Use the median value to replace the missing values in the mileage column
df['mileage'] = df['mileage'].fillna(df['mileage'].median())

print(df['mileage'].isnull().sum())

df

181
0


,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,seller,body_type,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country
0,alfa romeo,159,5500.0,233685.0,diesel,red,manual,136.0,1956.0,dealer,sedan,4.0,5.0,front,134.0,used,grey,NaN,2011.0,it
1,alfa romeo,159,13990.0,134000.0,gasoline,red,manual,200.0,1742.0,dealer,sedan,4.0,5.0,front,NaN,used,NaN,NaN,2010.0,es
2,renault,clio,2300.0,27000.0,gasoline,yellow,manual,80.0,800.0,dealer,sedan,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.0,be
3,opel,antara,3499.0,212575.0,gasoline,black,manual,140.0,2405.0,dealer,off-road/pick-up,5.0,5.0,4wd,229.0,used,NaN,metallic,2008.0,nl
4,alfa romeo,giulia,39800.0,7100.0,gasoline,white,automatic,280.0,1995.0,dealer,sedan,4.0,4.0,front,NaN,employee's car,full leather,black,2023.0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15412,tesla,model 3,31000.0,24000.0,electric,white,automatic,120.0,NaN,private seller,sedan,5.0,5.0,rear,NaN,used,full leather,black,2020.0,it
15413,tesla,model 3,28590.0,49393.0,electric,white,automatic,325.0,NaN,dealer,sedan,4.0,5.0,rear,NaN,used,NaN,metallic,2020.0,es
15414,tesla,model y,57900.0,7749.0,electric,black,automatic,534.0,NaN,private seller,off-road/pick-up,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,nl
15415,tesla,model y,59900.0,2500.0,electric,grey,NaN,480.0,NaN,dealer,sedan,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,fr


In [11]:
# Check if fuel_type column has any missing values
print(df['fuel_type'].isnull().sum())

# Replace the missing values with the most frequent value for the corresponding model 

print(df['fuel_type'].isnull().sum())

IndexError: single positional indexer is out-of-bounds

In [9]:
# Check if color column has any missing values
print(df['color'].isnull().sum())

# Replace the missing values with the most frequent value for the color column
df['color'] = df['color'].fillna(df['color'].mode()[0])

print(df['color'].isnull().sum())

110
0


In [13]:
# Step 1: Calculate the most common gearbox for each model
most_common_gearbox = df.groupby('model')['gearbox'].agg(pd.Series.mode)

# Step 2: Define a function to apply to each row which fills missing gearbox values
def fill_gearbox(row):
    if pd.isna(row['gearbox']):
        return most_common_gearbox[row['model']]
    else:
        return row['gearbox']

# Step 3: Apply the function to fill missing gearbox values
df['gearbox'] = df.apply(fill_gearbox, axis=1)

# Check the result
print(df['gearbox'].isnull().sum())


/var/folders/d8/m1_4xgr97g573bqjryrsl9yc0000gn/T/ipykernel_1004/1802135070.py:6: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(row['gearbox']):


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
# Check if power column has any missing values
print(df['power'].isnull().sum())

# Step 1: Calculate the median power for each model
median_power = df.groupby('model')['power'].median()

# Step 2: Define a function to apply to each row which fills missing power values
def fill_power(row):
    if pd.isna(row['power']):
        return median_power[row['model']]
    else:
        return row['power']

# Step 3: Apply the function to fill missing power values
df['power'] = df.apply(fill_power, axis=1)

# Check the result
print(df['power'].isnull().sum())

236
8


,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,seller,body_type,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country
0,alfa romeo,159,5500.0,233685.0,diesel,red,manual,136.0,1956.0,dealer,sedan,4.0,5.0,front,134.0,used,grey,NaN,2011.0,it
1,alfa romeo,159,13990.0,134000.0,gasoline,red,manual,200.0,1742.0,dealer,sedan,4.0,5.0,front,NaN,used,NaN,NaN,2010.0,es
2,renault,clio,2300.0,27000.0,gasoline,yellow,manual,80.0,800.0,dealer,sedan,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.0,be
3,opel,antara,3499.0,212575.0,gasoline,black,manual,140.0,2405.0,dealer,off-road/pick-up,5.0,5.0,4wd,229.0,used,NaN,metallic,2008.0,nl
4,alfa romeo,giulia,39800.0,7100.0,gasoline,white,automatic,280.0,1995.0,dealer,sedan,4.0,4.0,front,NaN,employee's car,full leather,black,2023.0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15412,tesla,model 3,31000.0,24000.0,electric,white,automatic,120.0,NaN,private seller,sedan,5.0,5.0,rear,NaN,used,full leather,black,2020.0,it
15413,tesla,model 3,28590.0,49393.0,electric,white,automatic,325.0,NaN,dealer,sedan,4.0,5.0,rear,NaN,used,NaN,metallic,2020.0,es
15414,tesla,model y,57900.0,7749.0,electric,black,automatic,534.0,NaN,private seller,off-road/pick-up,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,nl
15415,tesla,model y,59900.0,2500.0,electric,grey,automatic,480.0,NaN,dealer,sedan,5.0,5.0,4wd,NaN,used,full leather,black,2023.0,fr


In [18]:
# For the remaining missing values in the power column, replace them with the median power of the brand
df['power'] = df.groupby('brand')['power'].transform(lambda x: x.fillna(x.median()))

print(df['power'].isnull().sum())

0


In [20]:
# Check if engine_size column has any missing values
print(df['engine_size'].isnull().sum())

# Step 1: Calculate the median engine_size for each model
median_engine_size = df.groupby('model')['engine_size'].median()

# Step 2: Define a function to apply to each row which fills missing engine_size values
def fill_engine_size(row):
    if pd.isna(row['engine_size']):
        return median_engine_size[row['model']]
    else:
        return row['engine_size']

# Step 3: Apply the function to fill missing engine_size values
df['engine_size'] = df.apply(fill_engine_size, axis=1)

# Check the result
print(df['engine_size'].isnull().sum())

1626
136


In [22]:
# For the remaining missing values in the engine_size column, replace them with the median engine_size of the brand
df['engine_size'] = df.groupby('brand').groupby('brand')['engine_size'].transform(lambda x: x.fillna(x.median()))

print(df['engine_size'].isnull().sum())

0


In [25]:
# Check if seller column has any missing values
print(df['seller'].isnull().sum())

0


In [27]:
# Check if body_type column has any missing values
print(df['body_type'].isnull().sum())

0


In [32]:
# Check if doors column has any missing values
print(df['doors'].isnull().sum())

# Step 1: Calculate the most common doors for each model 

# Step 2: Define a function to apply to each row which fills missing doors values
def fill_doors(row):
    if pd.isna(row['doors']):
        return most_common_doors[row['model']]
    else:
        return row['doors']

# Step 3: Apply the function to fill missing doors values
df['doors'] = df.apply(fill_doors, axis=1)

print(df['doors'].isnull().sum())

4264


ValueError: Must produce aggregated value